# Run AML code in your custom Conda Environment from base image

Create and register my Environment with custom Conda and python packages from my docker image

## Check if environment already exists

In [ ]:
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core import Workspace
from azureml.core import Experiment, ScriptRunConfig, Environment

# Name of your Environment
myEnvironmentName = 'demoEnvFromDockerImageWithCustomCondaPythonPackages'

# Get the Workspace
ws = Workspace.from_config()
print(ws.name, "loaded")

try:
    env_name = Environment.get(workspace=ws, name = myEnvironmentName)
    print('Environment {} already exists, stop program'.format(env_name))
except Exception as e:
    print('Environment {} not found. Proceed to create it...'.format(myEnvironmentName))

# Create your custom environment using your docker image from your private registry

In [4]:
# Create the environment
env_name = Environment(name=myEnvironmentName)
# Enable Docker and reference an image
env_name.docker.enabled = True
# To use your own installed packages and disable Conda, set the parameter
env_name.python.user_managed_dependencies=True
env_name.docker.base_image = "<your azure container registry name>.azurecr.io/demo-aml/<name of the docker image>:latest"
env_name.docker.base_image_registry.username = "<azure container registry service principal id>"
env_name.docker.base_image_registry.password = "<service principal password>"


# Create or attach a compute target

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Name of the compute target
cluster_name = "CustomAMLCompute"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target.')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS3_V2',
                                                           max_nodes=4)
    # Create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    compute_target.wait_for_completion(show_output=True)

# Use get_status() to get a detailed status for the current AmlCompute.
print(compute_target.get_status().serialize())

# Now Train the model from your python script

In [ ]:
from azureml.widgets import RunDetails

training_folder = 'diabetes-training'
experiment_name = 'train-diabetes-in-custom-environment'

# Create a script config
script_config = ScriptRunConfig(source_directory=training_folder,
                                script='diabetes_training.py',
                                compute_target=compute_target,
                                environment=env_name) 

# submit the experiment run
experiment = Experiment(workspace=ws, name=experiment_name)
run = experiment.submit(config=script_config)

# Show the running experiment run in the notebook widget
RunDetails(run).show()

# Block until the experiment run has completed
run.wait_for_completion(show_output=True)

# Get logged metrics
metrics = run.get_metrics()
for key in metrics.keys():
        print(key, metrics.get(key))
print('\n')
for file in run.get_file_names():
    print(file)

# Register the custom environment

In [ ]:
env_name.register(workspace=ws)

# List all environments in the workspace

In [ ]:
Environment.list(workspace=ws)